<a href="https://colab.research.google.com/github/fatemehes75/SpaceX-Falcon-9/blob/main/SQL_spaceX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.


Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.

In [ ]:
!pip install sqlalchemy==1.3.9

### Connect to SQLlite db

In [ ]:
%load_ext sql
import csv, sqlite3

con = sqlite3.connect("spaceX_data1.db")
cur = con.cursor()

In [ ]:
%sql sqlite:///spaceX_data1.db

In [ ]:
import pandas as pd
df = pd.read_csv("Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [ ]:
#remove blank rows from table
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///spaceX_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: http://sqlalche.me/e/e3q8)


The names of the unique launch sites in the space mission

In [ ]:
%sql SELECT DISTINCT "Launch_Site" FROM SPACEXTABLE;

 * sqlite:///spaceX_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


5 records where launch sites begin with the string 'CCA'

In [ ]:
%sql SELECT * FROM SPACEXTABLE WHERE "Launch_Site" LIKE 'CCA%' LIMIT 5;

 * sqlite:///spaceX_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


The total payload mass carried by boosters launched by NASA (CRS)

In [ ]:
%sql SELECT "Customer", SUM("PAYLOAD_MASS__KG_") AS total_payload_mass FROM SPACEXTABLE GROUP BY "Customer" HAVING "Customer" = 'NASA (CRS)';

 * sqlite:///spaceX_data1.db
Done.


Customer,total_payload_mass
NASA (CRS),45596


Average payload mass carried by booster version F9 v1.1

In [ ]:
%sql SELECT "Booster_Version", AVG("PAYLOAD_MASS__KG_") AS average_payload_mass FROM SPACEXTABLE WHERE "Booster_Version" = 'F9 v1.1';

 * sqlite:///spaceX_data1.db
Done.


Booster_Version,average_payload_mass
F9 v1.1,2928.4


The date when the first succesful landing outcome in ground pad was acheived

In [ ]:
%sql SELECT MIN("Date") AS first_successful_landing_date  \
FROM SPACEXTABLE WHERE "Landing_Outcome" = 'Success (ground pad)';

 * sqlite:///spaceX_data1.db
Done.


first_successful_landing_date
2015-12-22


The names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [ ]:
%sql SELECT "Booster_Version","PAYLOAD_MASS__KG_" FROM  \
SPACEXTABLE WHERE "Landing_Outcome" = 'Success (drone ship)' AND "PAYLOAD_MASS__KG_" > 4000 AND "PAYLOAD_MASS__KG_" < 6000;


 * sqlite:///spaceX_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 FT B1022,4696
F9 FT B1026,4600
F9 FT B1021.2,5300
F9 FT B1031.2,5200


The total number of successful and failure mission outcomes

In [ ]:
%sql SELECT "Mission_Outcome", COUNT(*) AS total_count FROM SPACEXTABLE  \
GROUP BY "Mission_Outcome";

 * sqlite:///spaceX_data1.db
Done.


Mission_Outcome,total_count
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


The   names of the booster_versions which have carried the maximum payload mass.

In [ ]:
%sql SELECT "Booster_Version" FROM SPACEXTABLE WHERE  \
"PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_")FROM SPACEXTABLE);


 * sqlite:///spaceX_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015

In [ ]:
%sql SELECT SUBSTR("Date", 4, 2) AS month_name,"Landing_Outcome","Booster_Version","Launch_Site"  \
FROM SPACEXTABLE WHERE SUBSTR("Date", 7, 4) = '2015'AND "Landing_Outcome" = 'Failure (drone ship)';

 * sqlite:///spaceX_data1.db
Done.


month_name,Landing_Outcome,Booster_Version,Launch_Site


Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [ ]:
%sql SELECT "Landing_Outcome", COUNT(*) AS outcome_count FROM SPACEXTABLE \
WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20'  \
GROUP BY "Landing_Outcome"  \
ORDER BY outcome_count DESC;

 * sqlite:///spaceX_data1.db
Done.


Landing_Outcome,outcome_count
No attempt,10
Success (ground pad),5
Success (drone ship),5
Failure (drone ship),5
Controlled (ocean),3
Uncontrolled (ocean),2
Precluded (drone ship),1
Failure (parachute),1
